In [11]:
import cv2
import numpy as np
import imutils
from matplotlib import pyplot as plt


class Stitcher:
    @classmethod
    def drawMatches(cls, imageA, imageB, kpsA, kpsB, matches, status):
        # initialize the output visualization image
        (hA, wA) = imageA.shape[:2]
        (hB, wB) = imageB.shape[:2]
        vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
        vis[0:hA, 0:wA] = imageA
        vis[0:hB, wA:] = imageB
        # loop over the matches
        for ((trainIdx, queryIdx), s) in zip(matches, status):
            # only process the match if the keypoint was successfully
            # matched
            if s == 1:
                # draw the match
                ptA = (int(kpsA[queryIdx].pt[0]), int(kpsA[queryIdx].pt[1]))
                ptB = (int(kpsB[trainIdx].pt[0]) + wA, int(kpsB[trainIdx].pt[1]))
                cv2.line(vis, ptA, ptB, (0, 255, 0), 1)
        return vis

    @classmethod
    def keymatch(cls, match):
        dist1 = np.abs(1-(match[0].distance / match[1].distance))
        return dist1
    @classmethod
    def matchKeypoints(cls, kpsA, kpsB, featuresA, featuresB):
        # compute the raw matches and initialize the list of actual
        # matches
        matcher = cv2.DescriptorMatcher_create("BruteForce")
        rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
        rawMatches = sorted(rawMatches, key=cls.keymatch)
        #for m in rawMatches:
            #print(m[0].distance)
            #print(m[1].distance)
            #print('------------------')
        matches = []
        # loop over the raw matches

        for m in rawMatches:
            # ensure the distance is within a certain ratio of each
            # other (i.e. Lowe's ratio test)
            #print(m[0].distance / m[1].distance)
            #print(m)
            if 0.7 < m[0].distance / m[1].distance < 1.3:
                matches.append((m[0].trainIdx, m[0].queryIdx))


            # computing a homography requires at least 4 matches
        if len(matches) > 4:
            # construct the two sets of points
            #print(kpsA[0].pt)
            ptsA = np.float32([kpsA[i].pt for (_, i) in matches])
            ptsB = np.float32([kpsB[i].pt for (i, _) in matches])
            #print(ptsA)
            #print(ptsB)
            # compute the homography between the two sets of points
            (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC,4)
            #print(H[:2][:2])
            #print(H[:2, :2])
            #print(np.linalg.det(H[:2, :2]))
            # return the matches along with the homograpy matrix
            # and status of each matched point
            return (matches, H, status)

        # otherwise, no homograpy could be computed
        return None

    @classmethod
    def detectAndDescribe(cls, image):
        # convert the image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # detect and extract features from the image
        descriptor = cv2.xfeatures2d.SIFT_create()
        (kps, features) = descriptor.detectAndCompute(image, None)
        return (kps, features)

    @classmethod
    def stitch(cls, images, showMatches=False):
        # unpack the images, then detect keypoints and extract
        # local invariant descriptors from them
        (imageA, imageB) = images
        (kpsA, featuresA) = cls.detectAndDescribe(imageA)
        (kpsB, featuresB) = cls.detectAndDescribe(imageB)
        # match features between the two images
        M = cls.matchKeypoints(kpsA, kpsB,featuresA, featuresB)
        # if the match is None, then there aren't enough matched
        # keypoints to create a panorama
        if M is None:
            return None
        # otherwise, apply a perspective warp to stitch the images together
        (matches, H, status) = M
        result = cv2.warpPerspective(imageA, H, (imageA.shape[1] + imageB.shape[1], imageA.shape[0]))
        #result[0:imageB.shape[0], 0:imageB.shape[1]] = imageB

        # check to see if the keypoint matches should be visualized
        if showMatches:
            vis = cls.drawMatches(imageA, imageB, kpsA, kpsB, matches, status)

            # return a tuple of the stitched image and the
            # visualization
            return (result, vis)

        # return the stitched image
        return (result, None)

imageA = cv2.imread('carmel-00.png')
imageB = cv2.imread('carmel-01.png')
imageA = imutils.resize(imageA, width=600)
imageB = imutils.resize(imageB, width=600)
(kpsA, ftsA) = Stitcher.detectAndDescribe(imageA)
(kpsB, ftsB) = Stitcher.detectAndDescribe(imageB)
#a = Stitcher.stitch([imageA, imageB], showMatches=True)
# if a is not None:
#     cv2.imwrite('out.png', a[0])
#     cv2.imwrite('vis.png', a[1])
kpsImgA = None
kpsImgB = None
kpsImgA=cv2.drawKeypoints(imageA, kpsA, kpsImgA)
kpsImgB=cv2.drawKeypoints(imageB, kpsB, kpsImgB)
cv2.imwrite('kpsA.png', kpsImgA)
cv2.imwrite('kpsB.png', kpsImgB)

True

In [50]:
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_L1,crossCheck=False)
matches = bf.match(ftsA,ftsB)
matches = sorted(matches, key=lambda x:x.distance)
# for m in matches[:10]:
#     print(m.distance)
matchImg = None
matchImg = cv2.drawMatches(imageA, kpsA, imageB, kpsB, matches[:10], matchImg)
cv2.imwrite('matched.png', matchImg)
matchesIdx = [(m.queryIdx, m.trainIdx) for m in matches[:10]]
matchesCoordA = np.float32([kpsA[mind[0]].pt for mind in matchesIdx])
matchesCoordB = np.float32([kpsB[mind[1]].pt for mind in matchesIdx])


In [64]:
(H, mask) = cv2.findHomography(matchesCoordA, matchesCoordB, cv2.RANSAC)
np.linalg.det(H[:2, :2])
# B=cv2.getPerspectiveTransform(matchesCoordA[:4], matchesCoordB[:4])
# np.linalg.det(B)
imgDest = None
imgDest = cv2.warpPerspective(imageA, H, (imageA.shape[0]+imageB.shape[0], imageB.spahe[1]))
imgDest[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
cv2.imwrite('warped.png', imgDest)

True